In [2]:
import pandas as pd
from tqdm import tqdm
import sklearn
import pickle
from tqdm import tqdm
import time

from konlpy.tag import Kkma

In [3]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Bidirectional, LSTM, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam, SGD


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
import tqdm
import pickle
import copy

import sklearn
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import log_loss, accuracy_score,f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings(action='ignore')

import joblib

In [4]:
train = pd.read_table("train.txt", sep="|",encoding = "cp949")
sub = pd.read_csv("sample_submission.csv", encoding="cp949")
sub

In [5]:
train = train.fillna("")

train['total_text'] = train['text_obj'] + " " + train['text_mthd'] + " " + train['text_deal']
train.head()

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,total_text
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차부분정비 타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서 일반인을 대상으로 채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반소비자에게 열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학전아동보육


In [6]:
train_data = train.sort_values("digit_3")

In [7]:
train_data

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,total_text
701424,id_0701425,A,1,11,사업장에서,식물재배,나무.식물,사업장에서 식물재배 나무.식물
50323,id_0050324,A,1,11,영농조합법인에서,작물재배,썬레드,영농조합법인에서 작물재배 썬레드
230728,id_0230729,A,1,11,버섯종균,종사 및 배양 재배,버섯 재배,버섯종균 종사 및 배양 재배 버섯 재배
626694,id_0626695,A,1,11,사업장에서,재배,포포나무 재배,사업장에서 재배 포포나무 재배
751521,id_0751522,A,1,11,식물원에서,"？,나무재배",화웨재배,"식물원에서 ？,나무재배 화웨재배"
...,...,...,...,...,...,...,...,...
609695,id_0609696,S,96,969,빨래방에서,세탁기계를 갖추고,빨래방제공,빨래방에서 세탁기계를 갖추고 빨래방제공
342621,id_0342622,S,96,969,세탁소에서,손님들의 세탁물을 세탁,세탁서비스,세탁소에서 손님들의 세탁물을 세탁 세탁서비스
773448,id_0773449,S,96,969,세탁소,일반인대상으로,드라이크리닝,세탁소 일반인대상으로 드라이크리닝
536032,id_0536033,S,96,969,사업장에서,일반인등에게,"묘지이장, 분양","사업장에서 일반인등에게 묘지이장, 분양"


In [8]:
# 전처리된 데이터 불러오기
import pickle
with open('./train_text_kkma_morphs불1.pickle', 'rb') as f:
    clean_train_text = pickle.load(f)
    
with open('./test_text_kkma_morphs불1.pickle', 'rb') as f:
    clean_test_text = pickle.load(f)

In [11]:
# 데이터를 인덱스 벡터로 바꾼 후 패딩 처리 함수

def index_vetorize(clean_train,clean_test, sen_num, train_label):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(clean_train) #단어기반 인코딩
    
    train_sequences = tokenizer.texts_to_sequences(clean_train) #텍스트를 시퀀스로 변환
    test_sequences = tokenizer.texts_to_sequences(clean_test)

    word_vocab = tokenizer.word_index # 단어 사전 형태
    word_vocab["<PAD>"] = 0
    vocab_size = len(word_vocab)+1
    
    MAX_SEQUENCE_LENGTH = sen_num # 문장 최대 길이 (대략적인 평균길이 8) # 8,9,10
    train_inputs = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post') # 학습 데이터를 벡터화, 패딩설정
    train_labels = np.array(train_label).reshape(-1,1) # 학습 데이터의 라벨
    
    test_inputs = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post') # 테스트 데이터를 벡터화
    
    return train_inputs, train_labels,test_inputs, MAX_SEQUENCE_LENGTH, vocab_size

In [12]:
# label one hot encoding
from sklearn.preprocessing import OneHotEncoder
def label_one_hot(train_labels):
    ohe = OneHotEncoder(sparse = False)
    y = ohe.fit_transform(train_labels)
    ohe_list = list(ohe.categories_[0]) # 원래값으로 변환하기 위해 list 추출
    return y, ohe_list

In [13]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', patience=5,    
    min_delta=0.0001,
    restore_best_weights=True,
)

In [14]:
def GRU_model(vocab_size, MAX_SEQUENCE_LENGTH, label_cnt, lr):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, MAX_SEQUENCE_LENGTH, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(200, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(200)),
    tf.keras.layers.Dense(label_cnt, activation='softmax'),
        ])
    return model

In [15]:
train_label = train['digit_3']
label_cnt = train_label.nunique()

In [16]:
sen_num = 13
train_inputs, train_labels,test_inputs, MAX_SEQUENCE_LENGTH, vocab_size = index_vetorize(clean_train_text,clean_test_text, sen_num, train_label)

In [26]:
ohe_label , ohe_list = label_one_hot(train_labels)

In [25]:
# model 개발
lr=0.002
model_1 = GRU_model(vocab_size, MAX_SEQUENCE_LENGTH, label_cnt, lr)
model_1.compile(loss='categorical_crossentropy', optimizer=Adam(lr), metrics=['accuracy'])
history = model_1.fit(train_inputs, ohe_label,validation_split=0.2 ,
                      callbacks=early_stopping, batch_size=32, epochs=15)
model_1.save("GRU_200_sennum_13.h5")

Epoch 1/15
25000/25000 [==============================] - 450s 18ms/step - loss: 0.4926 - accuracy: 0.8763 - val_loss: 0.3838 - val_accuracy: 0.9008
Epoch 2/15
25000/25000 [==============================] - 449s 18ms/step - loss: 0.3312 - accuracy: 0.9121 - val_loss: 0.3569 - val_accuracy: 0.9075
Epoch 3/15
25000/25000 [==============================] - 449s 18ms/step - loss: 0.2956 - accuracy: 0.9202 - val_loss: 0.3494 - val_accuracy: 0.9084
Epoch 4/15
25000/25000 [==============================] - 448s 18ms/step - loss: 0.2730 - accuracy: 0.9252 - val_loss: 0.3480 - val_accuracy: 0.9093
Epoch 5/15
25000/25000 [==============================] - 447s 18ms/step - loss: 0.2564 - accuracy: 0.9289 - val_loss: 0.3575 - val_accuracy: 0.9093
Epoch 6/15
25000/25000 [==============================] - 449s 18ms/step - loss: 0.2432 - accuracy: 0.9324 - val_loss: 0.3599 - val_accuracy: 0.9093
Epoch 7/15
25000/25000 [==============================] - 449s 18ms/step - loss: 0.2333 - accuracy: 0.9347

In [32]:
from keras.models import load_model
model = load_model('GRU_200_sennum_13.h5')

In [33]:
digit3_pred = model.predict(test_inputs)

In [34]:
digit3_pred = np.argmax(digit3_pred, axis=1)

In [35]:
digit3_pred

array([147, 122, 219, ..., 132, 214, 169], dtype=int64)

In [36]:
sub['ohe'] = digit3_pred

In [37]:
sub['digit_3'] = sub['ohe'].map(lambda x : ohe_list[x])
sub

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,ohe
0,id_000001,I,56,561,치킨전문점에서,고객의주문에의해,치킨판매,147
1,id_000002,G,46,466,산업공구,다른 소매업자에게,철물 수공구,122
2,id_000003,S,94,949,절에서,신도을 대상으로,불교단체운영,219
3,id_000004,S,95,952,영업장에서,고객요구로,자동차튜닝,221
4,id_000005,I,56,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공",148
...,...,...,...,...,...,...,...,...
99995,id_099996,G,47,472,사업장에서,일반인대상으로,버섯농장,126
99996,id_099997,Q,86,862,한의원에서,외래환자위주고,치료,208
99997,id_099998,G,47,478,일반점포에서,소비자에게,그림판매,132
99998,id_099999,R,90,902,사업장에서,일반인.학생대상으로,학습공간제공,214


In [38]:
sub['digit_3'] = sub['digit_3'].map(lambda x : str(x))
sub['digit_2'] = sub['digit_3'].map(lambda x : x[:-1])
sub

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,ohe
0,id_000001,I,56,561,치킨전문점에서,고객의주문에의해,치킨판매,147
1,id_000002,G,46,466,산업공구,다른 소매업자에게,철물 수공구,122
2,id_000003,S,94,949,절에서,신도을 대상으로,불교단체운영,219
3,id_000004,S,95,952,영업장에서,고객요구로,자동차튜닝,221
4,id_000005,I,56,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공",148
...,...,...,...,...,...,...,...,...
99995,id_099996,G,47,472,사업장에서,일반인대상으로,버섯농장,126
99996,id_099997,Q,86,862,한의원에서,외래환자위주고,치료,208
99997,id_099998,G,47,478,일반점포에서,소비자에게,그림판매,132
99998,id_099999,R,90,902,사업장에서,일반인.학생대상으로,학습공간제공,214


In [39]:
digit_1_dict = {
    'A': ['1','2','3'],
    'B':['5','6','7','8'],
    'C':['10','11','12','13','14','15','16','17','18','19','20','21','22','23','24',
         '25','26','27','28','29','30','31','32','33','34'],
    'D':['35'],
    'E':['36','37','38','39'],
    'F':['41','42'],
    'G':['45','46','47'],
    'H':['49','50','51','52'],
    'I':['55','56'],
    'J':['58','59','60','61','62','63'],
    'K':['64','65','66'],
    'L':['68'],
    'N':['70','71','72','73'],
    'M':['74','75','76'],
    'O':['84'],
    'P':['85'],
    'Q':['86','87'],
    'R':['90',"91"],
    'S':['94','95','96'],
    'Y':['97','98'],
    'U': ['99'],
}

In [40]:
from tqdm import tqdm
for i in tqdm(range(len(sub['digit_2']))):
    for key, value in digit_1_dict.items():
        if sub['digit_2'][i] in value:
            sub['digit_1'][i] = key

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:29<00:00, 3400.91it/s]


In [41]:
sub

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,ohe
0,id_000001,I,56,561,치킨전문점에서,고객의주문에의해,치킨판매,147
1,id_000002,G,46,466,산업공구,다른 소매업자에게,철물 수공구,122
2,id_000003,S,94,949,절에서,신도을 대상으로,불교단체운영,219
3,id_000004,S,95,952,영업장에서,고객요구로,자동차튜닝,221
4,id_000005,I,56,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공",148
...,...,...,...,...,...,...,...,...
99995,id_099996,G,47,472,사업장에서,일반인대상으로,버섯농장,126
99996,id_099997,Q,86,862,한의원에서,외래환자위주고,치료,208
99997,id_099998,G,47,478,일반점포에서,소비자에게,그림판매,132
99998,id_099999,R,90,902,사업장에서,일반인.학생대상으로,학습공간제공,214


In [42]:
sub = sub.drop(['ohe'], axis=1)
sub

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56,561,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46,466,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94,949,절에서,신도을 대상으로,불교단체운영
3,id_000004,S,95,952,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,G,47,472,사업장에서,일반인대상으로,버섯농장
99996,id_099997,Q,86,862,한의원에서,외래환자위주고,치료
99997,id_099998,G,47,478,일반점포에서,소비자에게,그림판매
99998,id_099999,R,90,902,사업장에서,일반인.학생대상으로,학습공간제공


In [43]:
sub.to_csv("220413_GRU_200_sennum_13.csv", index = False)